<a href="https://colab.research.google.com/github/ace45800/eth-hash/blob/master/3commas_pyhton.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install ccxt requests

In [ ]:
import requests
import itertools
import time

# Configuration for Binance API
binance_api_base_url = 'https://api.binance.com'
binance_api_key = 'mYoXIRDlLaElluMpDf7OSjvbvanttnEwQNc5v4mlAz8BuaJaR9Y06ZqkoPfJVbu4'
binance_api_secret = 'w7C9Tzt5k3mjlbpv5Dum32ChP34iLcTolbmliEZGocsiJea5MDF28XohkakC2VNN'
bybit_api_base_url = 'https://api-testnet.bybit.com'
bybit_api_key = 'RxPYREEsR8fgGmnVqI'
bybit_api_secret = 'TrS5kXJjv3OvGAUvY3klhaAr9oOJ3nLuFqoH'

# Configuration for 3Commas API
three_commas_api_key = '5c9b0fe1c84b472d976e61dd4cc9878a254e95f5bd8c410bbcb68014021e1700'
three_commas_api_secret = '5e98a1c81515475e49ff8b55cb66b315cfc22e6bb016f9cad4ee7a89885847f59c482f6b64e1cc88474ffeb6cd649a5c939a365864c4d7eac2c768fcf280bde01934759450fb14e94c282cbe92c2c8f313370d4971d69f18f3d25fced65e629633e0a1a7'
three_commas_api_base_url = 'https://api.3commas.io/public/api'

In [ ]:
def get_bybit_pairs():
    url = f"{bybit_api_base_url}/v5/market/tickers"
    response = requests.get(url)
    print(response)
    data = response.json()
    print(data)
    pairs = [f"{symbol['base_currency']}/{symbol['quote_currency']}" for symbol in data['result']]
    return pairs

# Retrieve available trading pairs on Bybit
available_pairs = get_bybit_pairs()

<Response [403]>


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
def get_binance_pairs():
    url = f"{binance_api_base_url}/api/v3/exchangeInfo"
    response = requests.get(url)
    data = response.json()
    print(data)
    pairs = [symbol['symbol'] for symbol in data['symbols'] if symbol['status'] == 'TRADING']
    return pairs

# Retrieve available trading pairs on Binance
available_pairs = get_binance_pairs()

{'code': 0, 'msg': "Service unavailable from a restricted location according to 'b. Eligibility' in https://www.binance.com/en/terms. Please contact customer service if you believe you received this message in error."}


KeyError: 'symbols'

In [ ]:
import ccxt
import requests
import itertools
import json
import time

# 'apiKey': 'RxPYREEsR8fgGmnVqI',  'secret': 'TrS5kXJjv3OvGAUvY3klhaAr9oOJ3nLuFqoH', '
# Configuration de l'exchange Bybit avec ccxt
exchange = ccxt.binance({
    'apiKey': 'mYoXIRDlLaElluMpDf7OSjvbvanttnEwQNc5v4mlAz8BuaJaR9Y06ZqkoPfJVbu4',
    'secret': 'w7C9Tzt5k3mjlbpv5Dum32ChP34iLcTolbmliEZGocsiJea5MDF28XohkakC2VNN',
    'options': {
        'adjustForTimeDifference': True
    },
    'rateLimit': 200
})

# Configuration de l'API 3Commas
three_commas_api_key = '5c9b0fe1c84b472d976e61dd4cc9878a254e95f5bd8c410bbcb68014021e1700'
three_commas_api_secret = '5e98a1c81515475e49ff8b55cb66b315cfc22e6bb016f9cad4ee7a89885847f59c482f6b64e1cc88474ffeb6cd649a5c939a365864c4d7eac2c768fcf280bde01934759450fb14e94c282cbe92c2c8f313370d4971d69f18f3d25fced65e629633e0a1a7'
# URL de base de l'API 3Commas
three_commas_api_base_url = 'https://api.3commas.io/public/api'

In [ ]:
def get_available_pairs(exchange):
    markets = exchange.load_markets()
    time.sleep(exchange.rateLimit / 1000)
    return list(markets.keys())

# Récupérer les paires de trading disponibles sur Bybit
available_pairs = get_available_pairs(exchange)

ExchangeNotAvailable: binance GET https://api.binance.com/sapi/v1/capital/config/getall?timestamp=1719245797891&recvWindow=10000&signature=0ce1bc171649d005dedeb33ab7384b966fb059d760024960f88e0a454b0c902d 451  {
  "code": 0,
  "msg": "Service unavailable from a restricted location according to 'b. Eligibility' in https://www.binance.com/en/terms. Please contact customer service if you believe you received this message in error."
}

In [ ]:
def find_arbitrage_triangles(pairs):
    triangles = []
    for pair1, pair2, pair3 in itertools.combinations(pairs, 3):
        base1, quote1 = pair1.split('/')
        base2, quote2 = pair2.split('/')
        base3, quote3 = pair3.split('/')

        # Vérifie que les paires forment un triangle valide
        if (base1 == quote2 and base2 == quote3 and base3 == quote1) or \
           (quote1 == base2 and quote2 == base3 and quote3 == base1):
            triangles.append((pair1, pair2, pair3))
    return triangles

# Trouver les triangles d'arbitrage possibles
arbitrage_triangles = find_arbitrage_triangles(available_pairs)

In [ ]:
def calculate_arbitrage_opportunity(exchange, triangles):
    opportunities = []
    for pair1, pair2, pair3 in triangles:
        try:
            # Récupérer les données de marché pour les paires
            ticker1 = exchange.fetch_ticker(pair1)
            ticker2 = exchange.fetch_ticker(pair2)
            ticker3 = exchange.fetch_ticker(pair3)

            # Extraire les prix bid/ask des tickers
            price1_bid = ticker1['bid']
            price1_ask = ticker1['ask']
            price2_bid = ticker2['bid']
            price2_ask = ticker2['ask']
            price3_bid = ticker3['bid']
            price3_ask = ticker3['ask']

            # Exemple de calcul pour arbitrage triangulaire
            if 'BTC/USDT' in pair1 and 'ETH/BTC' in pair2 and 'ETH/USDT' in pair3:
                implicit_price = price2_ask / price3_bid
                if implicit_price > price1_ask:
                    opportunities.append((pair1, pair2, pair3))
                    print(f"Arbitrage Opportunity: Buy {pair1}, Sell {pair2}, Sell {pair3}")

            # Ajoute plus de conditions pour d'autres triangles formés
            # Ajuste les comparaisons de prix en conséquence

        except Exception as e:
            print(f"Erreur lors du calcul des opportunités d'arbitrage pour {pair1}, {pair2}, {pair3}: {str(e)}")

    return opportunities

# Calculer les opportunités d'arbitrage pour les triangles trouvés
opportunities = calculate_arbitrage_opportunity(exchange, arbitrage_triangles)

In [ ]:
def send_signal_to_3commas(opportunity, api_key, api_secret):
    url = f"{three_commas_api_base_url}/ver1/bots/{bot_id}/start_new_deal"

    headers = {
        'APIKEY': api_key,
        'SECRET': api_secret,
        'Content-Type': 'application/json'
    }

    data = {
        # Exemple de paramètres pour 3Commas, ajuste selon tes besoins
        "bot_id": "YOUR_BOT_ID",
        "pair": opportunity,
        "skip_signal_checks": True
    }

    try:
        response = requests.post(url, headers=headers, json=data)
        if response.status_code == 200:
            print(f"Signal envoyé avec succès à 3Commas pour {opportunity}")
        else:
            print(f"Erreur d'envoi: {response.status_code}, {response.text}")
    except Exception as e:
        print(f"Exception lors de l'envoi du signal à 3Commas: {str(e)}")

# Exemple d'utilisation pour envoyer des signaux pour chaque opportunité détectée
for opportunity in opportunities:
    send_signal_to_3commas(opportunity, three_commas_api_key, three_commas_api_secret)